In [1]:
import os
import sys
module_path_dont_know = os.path.abspath(os.path.join('../COBRAS_dont_know'))
module_path_testing = os.path.abspath(os.path.join('../COBRAS_testing'))

if module_path_dont_know not in sys.path:
    sys.path.append(module_path_dont_know)
    print("module path of dont_know added")

if module_path_testing not in sys.path:
    sys.path.append(module_path_testing)
    print("module path of testing added")
    
from before_clustering.generate_folds import generate_folds_for_dataset
from clustering_algorithms.kmeans_fixed_representative import KmeansFixedRepresentative
from clustering_algorithms.kmeans_plus_fixed_representative import KmeansPlusFixedRepresentative
from cobras.cobras import COBRAS
from cobras.cobras_logger import COBRASLogger
from cobras.querier.labelquerier import LabelQuerier
from cobras.querier.weak_querier import WeakQuerier
from cobras.super_instances.superinstance_select_representative import SuperInstance_select_representative_Builder
from config import FOLD_RESULT_DIR, FIGURE_DIR
from evaluate_clusterings.calculate_aligned_rank import calculate_and_write_aligned_rank
from evaluate_clusterings.calculate_aris import calculate_n_times_n_fold_aris_for_testnames
from evaluate_clusterings.calculate_average_aris import calculate_average_aris
from generate_clusterings.clustering_task import make_n_run_10_fold_cross_validation
from heuristics.select_super_instance_heuristics import *
from heuristics.splitlevel_estimation_strategy import *
from pathlib import Path
from present_results.plot_aligned_rank import plot_rank_comparison_file
from present_results.plot_aris import plot_average_ARI_per_dataset
from present_results.plot_aris import plot_overall_average_ARI
from present_results.plot_query_reuse import calculate_and_plot_query_reuse
from run_locally.run_tests import run_clustering_tasks_locally
from util.datasets import Dataset

print("imports succeeded")

module path of dont_know added
module path of testing added
imports succeeded


In [2]:
if not Path(FOLD_RESULT_DIR).exists():
    generate_folds_for_dataset()
print("done")

done


In [3]:
splitstrat = StandardSplitLevelEstimationStrategyAlwayskmeans(SelectMostInstancesHeuristic())
clusterer_1 = COBRAS()
querier = LabelQuerier(None,100)
#querier = WeakQuerier(data.data,data.target,1000,'local_nondet')
#querier = WeakQuerier(None, None ,1000,'local_nondet')
test_name_1 = "standard_COBRAS"
all_dataset_names = Dataset.get_standard_dataset_names()#["iris"]#
clustering_tasks = make_n_run_10_fold_cross_validation(test_name_1, clusterer_1,querier,all_dataset_names,3)
print("done")

done


In [4]:
clusterer_2 = COBRAS(cluster_algo = KmeansFixedRepresentative())
test_name_2 = "2"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_2, clusterer_2, querier, all_dataset_names, 3))
print("done")

done


In [5]:
clusterer_3 = COBRAS(cluster_algo = KmeansPlusFixedRepresentative())
test_name_3 = "3"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_3, clusterer_3, querier, all_dataset_names, 3))
print("done")

done


In [6]:
clusterer_4 = COBRAS(cluster_algo = KmeansFixedRepresentative(), splitlevel_strategy = splitstrat)
test_name_4 = "4"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_4, clusterer_4, querier, all_dataset_names, 3))
print("done")

done


In [7]:
clusterer_5 = COBRAS(cluster_algo = KmeansPlusFixedRepresentative(), splitlevel_strategy=splitstrat)
test_name_5 = "5"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_5, clusterer_5, querier, all_dataset_names, 3))
print("done")

done


In [8]:
clusterer_6 = COBRAS(superinstance_builder= SuperInstance_select_representative_Builder())
test_name_6 = "6"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_6, clusterer_6, querier, all_dataset_names, 3))
print("done")

done


In [9]:
clusterer_7 = COBRAS(cluster_algo = KmeansFixedRepresentative(), superinstance_builder= SuperInstance_select_representative_Builder())
test_name_7 = "7"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_7, clusterer_7, querier, all_dataset_names, 3))
print("done")

done


In [10]:
clusterer_8 = COBRAS(cluster_algo = KmeansPlusFixedRepresentative(),superinstance_builder= SuperInstance_select_representative_Builder())
test_name_8 = "8"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_8, clusterer_8, querier, all_dataset_names, 3))
print("done")

done


In [11]:
clusterer_9 = COBRAS(cluster_algo = KmeansFixedRepresentative(),superinstance_builder= SuperInstance_select_representative_Builder(), splitlevel_strategy=splitstrat)
test_name_9 = "9"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_9, clusterer_9, querier, all_dataset_names, 3))
print("done")

done


In [12]:
clusterer_10 = COBRAS(cluster_algo = KmeansPlusFixedRepresentative(),superinstance_builder= SuperInstance_select_representative_Builder() , splitlevel_strategy=splitstrat)
test_name_10 = "10"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_10, clusterer_10, querier, all_dataset_names, 3))
print("done")

done


In [13]:
run_clustering_tasks_locally(clustering_tasks,4)
print("done")

100%|██████████| 5670/5670 [9:03:04<00:00,  5.75s/it]     


done


In [2]:
test_name_1 = "standard_COBRAS"
test_name_5 = "5"
test_name_6 = "6"
test_name_10 = "10"
test_names = [test_name_1, test_name_5, test_name_6, test_name_10]#,test_name_1, test_name_3, test_name_5, test_name_6, test_name_7, test_name_8, test_name_9, test_name_10]
comparison_name = 'presentation_comparison_1'
nb_of_cores = 4
query_budget = 100

#print(clusterer_1.logger.nr_reused_constraints)

calculate_n_times_n_fold_aris_for_testnames(test_names, nb_cores=nb_of_cores)
calculate_average_aris(test_names, query_budget)
calculate_and_write_aligned_rank(test_names,comparison_name)

plot_average_ARI_per_dataset(comparison_name, test_names, test_names)
plot_overall_average_ARI(comparison_name, test_names, test_names)
plot_rank_comparison_file(comparison_name, test_names, test_names)

calculate_and_plot_query_reuse(comparison_name, test_names)

print(f"all resulting plots are in {Path(FIGURE_DIR)/comparison_name}")

  0%|          | 0/4 [00:00<?, ?it/s]

Calculating ARIs for n-times n-fold:  ['standard_COBRAS', '5', '6', '10']
already calculated
Calculating average ARIs


 25%|██▌       | 1/4 [00:07<00:23,  7.82s/it]

less clusterings (44) than querybudget (100), correcting by repeating last ARI5
less clusterings (60) than querybudget (100), correcting by repeating last ARI5
less clusterings (69) than querybudget (100), correcting by repeating last ARI5
less clusterings (90) than querybudget (100), correcting by repeating last ARI5
less clusterings (56) than querybudget (100), correcting by repeating last ARI5
less clusterings (59) than querybudget (100), correcting by repeating last ARI5
less clusterings (85) than querybudget (100), correcting by repeating last ARI5
less clusterings (26) than querybudget (100), correcting by repeating last ARI5
less clusterings (99) than querybudget (100), correcting by repeating last ARI5
less clusterings (33) than querybudget (100), correcting by repeating last ARI5
less clusterings (57) than querybudget (100), correcting by repeating last ARI5
less clusterings (71) than querybudget (100), correcting by repeating last ARI5
less clusterings (90) than querybudget (

 75%|███████▌  | 3/4 [00:22<00:07,  7.59s/it]

less clusterings (98) than querybudget (100), correcting by repeating last ARI10
less clusterings (60) than querybudget (100), correcting by repeating last ARI10
less clusterings (67) than querybudget (100), correcting by repeating last ARI10
less clusterings (70) than querybudget (100), correcting by repeating last ARI10
less clusterings (94) than querybudget (100), correcting by repeating last ARI10
less clusterings (15) than querybudget (100), correcting by repeating last ARI10
less clusterings (47) than querybudget (100), correcting by repeating last ARI10
less clusterings (83) than querybudget (100), correcting by repeating last ARI10
less clusterings (49) than querybudget (100), correcting by repeating last ARI10
less clusterings (81) than querybudget (100), correcting by repeating last ARI10
less clusterings (75) than querybudget (100), correcting by repeating last ARI10
less clusterings (45) than querybudget (100), correcting by repeating last ARI10
less clusterings (54) than q

100%|██████████| 4/4 [00:30<00:00,  7.56s/it]


calculating average rank
0.07318382394465822
0.07856603636560741
0.07957117301489963
0.2007159151620091
all resulting plots are in \Users\nicol\Documents\KUL 2020-2021\thesis\code\results\results\figures\presentation_comparison_1
